# Amazon Bedrock AgentCore 런타임(Runtime)에서 Strands 에이전트(Agent)를 사용한 스트리밍(Streaming) 응답

## 개요

이 튜토리얼에서는 Amazon Bedrock AgentCore 런타임(Runtime)을 사용하여 스트리밍(Streaming) 응답을 구현하는 방법을 배웁니다. 이 예제는 부분 결과가 사용 가능해질 때 스트리밍(Streaming)하는 방법을 보여주어, 대량의 콘텐츠를 생성하거나 상당한 처리 시간이 걸리는 작업에 대해 더 반응적인 사용자 경험을 제공합니다.


### 튜토리얼 세부사항

|정보| 세부사항|
|:--------------------|:---------------------------------------------------------------------------------|
| 튜토리얼 유형       | 스트리밍(Streaming)이 포함된 대화형|
| 에이전트(Agent) 유형 | 단일         |
| 에이전틱 프레임워크(Agentic Framework)   | Strands Agents |
| LLM 모델           | Anthropic Claude Haiku 4.5 |
| 튜토리얼 구성 요소 | AgentCore 런타임(Runtime)과 Strands 에이전트(Agent) 및 Amazon Bedrock 모델을 사용한 스트리밍(Streaming) 응답 |
| 튜토리얼 분야   | 범용                                                                   |
| 예제 복잡도  | 쉬움                                                                             |
| 사용 SDK            | Amazon BedrockAgentCore Python SDK 및 boto3|

### 튜토리얼 아키텍처

이 튜토리얼에서는 스트리밍(Streaming) 에이전트(Agent)를 AgentCore 런타임(Runtime)에 배포(Deployment)하는 방법을 설명합니다.

데모 목적으로 스트리밍(Streaming) 기능이 있는 Amazon Bedrock 모델을 사용하는 Strands 에이전트(Agent)를 사용합니다.

예제에서는 `get_weather`와 `get_time` 두 가지 도구(Tool)를 가진 간단한 에이전트(Agent)를 사용하지만, 스트리밍(Streaming) 응답 기능을 갖추고 있습니다.

    
<div style="text-align:left">
    <img src="images/architecture_runtime.png" width="60%"/>
</div>

### 튜토리얼 주요 기능

* Amazon Bedrock AgentCore 런타임(Runtime)의 에이전트(Agent)에서 스트리밍(Streaming) 응답
* 실시간 부분 결과 전달
* 스트리밍(Streaming)과 함께 Amazon Bedrock 모델 사용
* 비동기 스트리밍(Streaming) 지원이 있는 Strands 에이전트(Agent) 사용

## 사전 요구사항

이 튜토리얼을 실행하려면 다음이 필요합니다:
* Python 3.10+
* AWS 자격 증명
* Amazon Bedrock AgentCore SDK
* Strands Agents
* Docker 실행 중

In [ ]:
!pip install --force-reinstall -U -r requirements.txt --quiet

## Preparing your streaming agent for deployment on AgentCore Runtime

Let's now deploy our streaming agents to AgentCore Runtime. The streaming functionality is handled automatically by the AgentCore SDK when you use async generators or yield statements in your entrypoint function.

Key points for streaming implementation:
* Use `async def` for your entrypoint function
* Use `yield` to stream chunks as they become available
* The AgentCore SDK automatically handles the Server-Sent Events (SSE) format
* Clients will receive Content-Type: text/event-stream responses

### Strands Agents with Amazon Bedrock model and Streaming
Let's look at our streaming implementation for the Strands Agent using Amazon Bedrock model.

In [ ]:
%%writefile strands_claude_streaming.py
from strands import Agent, tool
from strands_tools import calculator # Import the calculator tool
import argparse
import json
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from strands.models import BedrockModel
import asyncio
from datetime import datetime

app = BedrockAgentCoreApp()

# Create a custom tool 
@tool
def weather():
    """ Get weather """ # Dummy implementation
    return "sunny"

@tool
def get_time():
    """ Get current time """
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

model_id = "global.anthropic.claude-haiku-4-5-20251001-v1:0"
model = BedrockModel(
    model_id=model_id,
)
agent = Agent(
    model=model,
    tools=[
        calculator, weather, get_time
    ],
    system_prompt="""You're a helpful assistant. You can do simple math calculations, 
    tell the weather, and provide the current time."""
)

@app.entrypoint
async def strands_agent_bedrock_streaming(payload):
    """
    Invoke the agent with streaming capabilities
    This function demonstrates how to implement streaming responses
    with AgentCore Runtime using async generators
    """
    user_input = payload.get("prompt")
    print("User input:", user_input)
    
    try:
        # Stream each chunk as it becomes available
        async for event in agent.stream_async(user_input):
            if "data" in event:
                yield event["data"]
            
    except Exception as e:
        # Handle errors gracefully in streaming context
        error_response = {"error": str(e), "type": "stream_error"}
        print(f"Streaming error: {error_response}")
        yield error_response

if __name__ == "__main__":
    app.run()

## AgentCore 런타임(Runtime) 배포(Deployment)를 위한 스트리밍(Streaming) 에이전트(Agent) 준비

이제 스트리밍(Streaming) 에이전트(Agent)를 AgentCore 런타임(Runtime)에 배포(Deployment)해 보겠습니다. 스트리밍(Streaming) 기능은 진입점 함수에서 async 제너레이터나 yield 문을 사용할 때 AgentCore SDK에서 자동으로 처리됩니다.

스트리밍(Streaming) 구현의 핵심 포인트:
* 진입점 함수에 `async def` 사용
* 청크가 사용 가능해질 때 `yield`를 사용하여 스트리밍(Streaming)
* AgentCore SDK가 자동으로 Server-Sent Events(SSE) 형식 처리
* 클라이언트는 Content-Type: text/event-stream 응답을 받음

### Amazon Bedrock 모델과 스트리밍(Streaming)이 있는 Strands 에이전트(Agent)
Amazon Bedrock 모델을 사용하는 Strands 에이전트(Agent)의 스트리밍(Streaming) 구현을 살펴보겠습니다.

## Deploying the streaming agent to AgentCore Runtime

The `CreateAgentRuntime` operation supports comprehensive configuration options, letting you specify container images, environment variables and encryption settings. You can also configure protocol settings (HTTP, MCP) and authorization mechanisms to control how your clients communicate with the agent. 

**Note:** Operations best practice is to package code as container and push to ECR using CI/CD pipelines and IaC

In this tutorial we will use the Amazon Bedrock AgentCode Python SDK to easily package your artifacts and deploy them to AgentCore runtime.

## AgentCore 런타임(Runtime)에서의 스트리밍(Streaming) 이해

AgentCore 런타임(Runtime)에서 스트리밍(Streaming)을 사용할 때 여러 가지가 자동으로 처리됩니다:

### Server-Sent Events(SSE) 형식
* AgentCore SDK가 자동으로 yield된 데이터를 SSE 형식으로 변환
* 각 yield는 SSE 스트림에서 `data: ` 이벤트가 됨
* Content-Type이 자동으로 `text/event-stream`으로 설정됨

### 클라이언트 처리
* 클라이언트는 에이전트(Agent)가 요청을 처리할 때 실시간 업데이트를 받음
* 이를 통해 점진적 응답 표시와 더 나은 사용자 경험이 가능
* 클라이언트는 전체 응답이 준비되기 전에 부분 결과를 처리할 수 있음

### 오류 처리
* 스트리밍(Streaming) 응답에는 적절한 오류 처리가 포함되어야 함
* 오류는 스트림의 일부로 yield될 수 있음
* 함수가 완료되거나 처리되지 않은 예외가 발생하면 스트림이 종료됨

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
boto_session = Session()
region = boto_session.region_name
region

agentcore_runtime = Runtime()

response = agentcore_runtime.configure(
    entrypoint="strands_claude_streaming.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name="strands_claude_streaming"
)
response

## AgentCore 런타임(Runtime)에 스트리밍(Streaming) 에이전트(Agent) 배포(Deployment)

`CreateAgentRuntime` 작업은 포괄적인 구성(Configuration) 옵션을 지원하여 컨테이너 이미지, 환경 변수 및 암호화 설정을 지정할 수 있습니다. 또한 프로토콜 설정(HTTP, MCP)과 인가(Authorization) 메커니즘을 구성(Configuration)하여 클라이언트가 에이전트(Agent)와 통신하는 방식을 제어할 수 있습니다.

**참고:** 운영 모범 사례는 코드를 컨테이너로 패키징하고 CI/CD 파이프라인 및 IaC를 사용하여 ECR에 푸시하는 것입니다.

이 튜토리얼에서는 Amazon Bedrock AgentCode Python SDK를 사용하여 아티팩트를 쉽게 패키징하고 AgentCore 런타임(Runtime)에 배포(Deployment)할 수 있습니다.

In [ ]:
launch_result = agentcore_runtime.launch()

### AgentCore 런타임(Runtime) 배포(Deployment) 구성(Configuration)

다음으로 스타터 툴킷을 사용하여 진입점, 방금 생성한 실행 역할 및 requirements 파일로 AgentCore 런타임(Runtime) 배포(Deployment)를 구성(Configuration)합니다. 또한 시작 시 Amazon ECR 리포지토리를 자동 생성하도록 스타터 키트를 구성(Configuration)합니다.

구성(Configuration) 단계에서 애플리케이션 코드를 기반으로 도커 파일이 생성됩니다.

<div style="text-align:left">
    <img src="images/configure.png" width="60%"/>
</div>

In [ ]:
import time

status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

### Invoking AgentCore Runtime with Streaming

Finally, we can invoke our AgentCore Runtime with a payload and receive streaming responses

<div style="text-align:left">
    <img src="images/invoke.png" width="85%"/>
</div>

In [ ]:
invoke_response = agentcore_runtime.invoke({
    "prompt": 
    "what the weather is like?"
})
invoke_response

### AgentCore 런타임(Runtime)에 스트리밍(Streaming) 에이전트(Agent) 시작

이제 도커 파일이 준비되었으니 스트리밍(Streaming) 에이전트(Agent)를 AgentCore 런타임(Runtime)에 시작해 보겠습니다. 이렇게 하면 Amazon ECR 리포지토리와 AgentCore 런타임(Runtime)이 생성됩니다.

<div style="text-align:left">
    <img src="images/launch.png" width="85%"/>
</div>

In [ ]:
import boto3
import json
from IPython.display import Markdown, display

agent_arn = launch_result.agent_arn
agentcore_client = boto3.client(
    'bedrock-agentcore',
    region_name=region
)

# For streaming responses, we need to handle the EventStream
boto3_response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn=agent_arn,
    qualifier="DEFAULT",
    payload=json.dumps({"prompt": "How much is 2+1"})
)

# Check if the response is streaming
if "text/event-stream" in boto3_response.get("contentType", ""):
    print("Processing streaming response with boto3:")
    content = []
    for line in boto3_response["response"].iter_lines(chunk_size=1):
        if line:
            line = line.decode("utf-8")
            if line.startswith("data: "):
                data = line[6:].replace('"', '')  # Remove "data: " prefix
                print(f"Received streaming chunk: {data}")
                content.append(data.replace('"', ''))
    
    # Display the complete streamed response
    full_response = " ".join(content)
    display(Markdown(full_response))
else:
    # Handle non-streaming response
    try:
        events = []
        for event in boto3_response.get("response", []):
            events.append(event)
    except Exception as e:
        events = [f"Error reading EventStream: {e}"]
    
    if events:
        try:
            response_data = json.loads(events[0].decode("utf-8"))
            display(Markdown(response_data))
        except:
            print(f"Raw response: {events[0]}")

## Benefits of Streaming Responses

Streaming responses provide several key advantages:

### User Experience
* **Immediate Feedback**: Users see partial results as they become available
* **Perceived Performance**: Responses feel faster even if total time is the same
* **Progressive Display**: Long responses can be displayed incrementally

### Technical Benefits
* **Memory Efficient**: Process large responses without loading everything into memory
* **Timeout Prevention**: Avoid timeouts on long-running operations
* **Real-time Processing**: Handle real-time data as it becomes available

### Use Cases
* **Content Generation**: Long-form writing, reports, documentation
* **Data Analysis**: Progressive results from complex calculations
* **Multi-step Workflows**: Show progress through complex agent reasoning
* **Real-time Monitoring**: Live updates from monitoring agents

### AgentCore 런타임(Runtime) 상태 확인
AgentCore 런타임(Runtime)을 배포(Deployment)했으니 배포(Deployment) 상태를 확인해 보겠습니다.

In [ ]:
launch_result.ecr_uri, launch_result.agent_id, launch_result.ecr_uri.split('/')[1]

In [ ]:
agentcore_control_client = boto3.client(
    'bedrock-agentcore-control',
    region_name=region
)
ecr_client = boto3.client(
    'ecr',
    region_name=region
)

runtime_delete_response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_result.agent_id,
)

response = ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split('/')[1],
    force=True
)

# Congratulations!

You have successfully implemented and deployed a streaming agent using Amazon Bedrock AgentCore Runtime! 

## What you've learned:
* How to implement streaming responses using async generators
* How AgentCore Runtime automatically handles SSE format
* How to process streaming responses on the client side
* The benefits of streaming for user experience and performance

## Next steps:
* Experiment with different streaming patterns for your use cases
* Implement custom streaming logic for complex multi-step workflows
* Explore combining streaming with other AgentCore features like Memory and Gateway
* Consider implementing client-side streaming visualization for better UX